In [53]:
import pandas as pd
import openbabel
import numpy as np
import os
from tqdm import tqdm , tqdm_notebook
tqdm_notebook().pandas()

In [56]:
base_path = '/home/jovyan/work/Molecular_Properties'

In [57]:
structures = pd.read_csv(f'{base_path}/structures.csv')
# data reduction
train = pd.read_csv(f'{base_path}/train.csv')[::10]
test = pd.read_csv(f'{base_path}/test.csv')[::10]

In [58]:
obConversion = openbabel.OBConversion()
obConversion.SetInFormat("xyz")
xyz_path = f'{base_path}/Structures'

In [59]:
print(xyz_path)

/home/jovyan/work/Molecular_Properties/Structures


In [60]:
def cis_trans_bond_indices(molecule_name):
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, f'{xyz_path}/{molecule_name}.xyz')
    obs = openbabel.OBStereoFacade(mol)
    has_ct = [obs.HasCisTransStereo(n) for n in range(mol.NumBonds())]
    return [i for i, x in enumerate(has_ct) if x == True] if has_ct else []

In [63]:
df = pd.DataFrame(structures.molecule_name.unique(), columns=['molecule_name'])
df.head(10)

,molecule_name
0,dsgdb9nsd_000001
1,dsgdb9nsd_000002
2,dsgdb9nsd_000003
3,dsgdb9nsd_000004
4,dsgdb9nsd_000005
5,dsgdb9nsd_000007
6,dsgdb9nsd_000008
7,dsgdb9nsd_000009
8,dsgdb9nsd_000010
9,dsgdb9nsd_000011


In [64]:
df['bond_indices'] = df.molecule_name.progress_apply(lambda x: cis_trans_bond_indices(x))
df['len_bond_indices'] = df.bond_indices.progress_apply(lambda x:len(x))

In [68]:
train = pd.merge(train, df, how='left', on='molecule_name')
test = pd.merge(test, df, how='left', on='molecule_name')

In [69]:
def is_cis_trans(molecule_name, bond_indices, atom_index_0, atom_index_1):
    if len(bond_indices) == 0:
        return pd.Series([0,0])

    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, f'{xyz_path}/{molecule_name}.xyz')
    obs = openbabel.OBStereoFacade(mol)
    
    is_cis   = [obs.GetCisTransStereo(i).IsCis(atom_index_0, atom_index_1) for i in bond_indices]
    is_trans = [obs.GetCisTransStereo(i).IsTrans(atom_index_0, atom_index_1) for i in bond_indices]
    return pd.Series([int(True in is_cis), int(True in is_trans)])

In [70]:
train[['is_cis','is_trans']] = train.progress_apply(lambda x: is_cis_trans(x.molecule_name,
                                                                           x.bond_indices,
                                                                           x.atom_index_0,
                                                                           x.atom_index_1), axis=1)

In [77]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,bond_indices,len_bond_indices,is_cis,is_trans
0,0,dsgdb9nsd_000001,1,0,1JHC,84.80760,[],0,0,0
1,10,dsgdb9nsd_000002,1,0,1JHN,32.68890,[],0,0,0
2,20,dsgdb9nsd_000007,2,1,2JHC,-2.37831,[],0,0,0
3,30,dsgdb9nsd_000007,3,6,3JHH,3.25253,[],0,0,0
4,40,dsgdb9nsd_000007,5,7,2JHH,-11.69930,[],0,0,0


In [73]:
test[['is_cis','is_trans']] = test.progress_apply(lambda x: is_cis_trans(x.molecule_name,
                                                                        x.bond_indices,
                                                                        x.atom_index_0,
                                                                        x.atom_index_1), axis=1)



In [75]:
angles = pd.read_csv(f'{base_path}/angles.csv')

In [76]:
angles.head()

,molecule_name,atom_index_0,atom_index_1,shortest_path_atoms,shortest_path_n_bonds,cosinus,dihedral
0,dsgdb9nsd_000001,1,0,NaN,1,NaN,NaN
1,dsgdb9nsd_000001,1,2,C,2,-0.33329,NaN
2,dsgdb9nsd_000001,1,3,C,2,-0.33333,NaN
3,dsgdb9nsd_000001,1,4,C,2,-0.33335,NaN
4,dsgdb9nsd_000001,2,0,NaN,1,NaN,NaN
